# First notebook with Python3.11
## Load the qonnx model and convert it to an hls model

In [1]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes

In [2]:
model_path = "qonnx_fragm_model.onnx"
qonnx_model = ModelWrapper(model_path)
qonnx_model = qonnx_model.transform(InferShapes())

### Convert to an HLS model

In [3]:
from hls4ml.utils.config import config_from_onnx_model
from hls4ml.converters import convert_from_onnx_model

In [4]:
hls_config = config_from_onnx_model(
    qonnx_model,
    granularity='name',
    backend='Vitis',
    default_precision='fixed<16,6>'
)

Output layers:  ['Identity_1']
Input shape: [57]
Topology:
Layer name: Quant_0, layer type: Quant, current shape: [[9, 10]]
Layer name: Quant_1, layer type: Quant, current shape: [[10]]
Layer name: Quant_2, layer type: Quant, current shape: [[6, 9]]
Layer name: Quant_3, layer type: Quant, current shape: [[9]]
Layer name: Quant_4, layer type: Quant, current shape: [[4, 6]]
Layer name: Quant_5, layer type: Quant, current shape: [[6]]
Layer name: Quant_6, layer type: Quant, current shape: [[6, 4]]
Layer name: Quant_7, layer type: Quant, current shape: [[4]]
Layer name: Quant_8, layer type: Quant, current shape: [[9, 6]]
Layer name: Quant_9, layer type: Quant, current shape: [[6]]
Layer name: Quant_10, layer type: Quant, current shape: [[10, 9]]
Layer name: Quant_11, layer type: Quant, current shape: [[9]]
Layer name: Quant_12, layer type: Quant, current shape: [[29, 10]]
Layer name: Quant_13, layer type: Quant, current shape: [[10]]
Layer name: Quant_14, layer type: Quant, current shape: 

In [5]:
hls_config

{'Model': {'Precision': {'default': 'fixed<16,6>'},
  'ReuseFactor': 1,
  'Strategy': 'Latency',
  'BramFactor': 1000000000,
  'TraceOutput': False},
 'LayerName': {'global_in': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_0_param1': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_0_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_1_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_0_param3': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_2_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_1_param3': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_3_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_16_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_4_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_5_param0': {'Trace': False, 'Precision': {'result': 'auto'}},
  'Quant_6_param0': {'Trace': False, 'Precision': {'result':

Custom flows

In [8]:
hls_config['Flows'] = {'parse_qonnx'}

In [9]:
qonnx_model = qonnx_model.cleanup()
hls_model = convert_from_onnx_model(
    qonnx_model,
    output_dir='hls_model8',
    io_type='io_stream',    # default is 'io_parallel'
    backend='Vitis',
    hls_config=hls_config,
)

Interpreting Model ...
Output layers:  ['Identity_1']
Input shape: [57]
Topology:
Layer name: Quant_0, layer type: Quant, current shape: [[9, 10]]
Layer name: Quant_1, layer type: Quant, current shape: [[10]]
Layer name: Quant_2, layer type: Quant, current shape: [[6, 9]]
Layer name: Quant_3, layer type: Quant, current shape: [[9]]
Layer name: Quant_4, layer type: Quant, current shape: [[4, 6]]
Layer name: Quant_5, layer type: Quant, current shape: [[6]]
Layer name: Quant_6, layer type: Quant, current shape: [[6, 4]]
Layer name: Quant_7, layer type: Quant, current shape: [[4]]
Layer name: Quant_8, layer type: Quant, current shape: [[9, 6]]
Layer name: Quant_9, layer type: Quant, current shape: [[6]]
Layer name: Quant_10, layer type: Quant, current shape: [[10, 9]]
Layer name: Quant_11, layer type: Quant, current shape: [[9]]
Layer name: Quant_12, layer type: Quant, current shape: [[29, 10]]
Layer name: Quant_13, layer type: Quant, current shape: [[10]]
Layer name: Quant_14, layer type:

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv_3_11/lib64/python3.11/site-packages/hls4ml/model/optimizer/passes/move_scales.py:93: UserWarning: Failed to propagate quantization scales down MatMul node; model probably not suppored.
  warnings.warn(


### Look at the uncompiled HLS model

In [10]:
hls_model._applied_flows

[{'parse_qonnx': {'fuse_quant_with_constant',
   'matmul_const_to_dense',
   'merge_to_apply_alpha',
   'quant_constant_parameters',
   'quant_to_activation',
   'scale_down_mat_mul'}}]

In [ ]:
layer_dicts = []

In [ ]:
for _layer_name in hls_model.graph.keys():
    layer_dicts.append(hls_model.graph.get(_layer_name).__dict__)

In [ ]:
hls_model.graph

In [ ]:
dir(hls_model.graph.get(_layer_name))

In [ ]:
layer_dicts[4]